In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('PartD_Prescriber_PUF_NPI_17.txt',sep='\t')
print(data.head())

          npi nppes_provider_last_org_name nppes_provider_first_name  \
0  1003000126                    ENKESHAFI                   ARDALAN   
1  1003000142                       KHALIL                    RASHID   
2  1003000167                      ESCOBAR                     JULIO   
3  1003000175                REYES-VASQUEZ                   BELINDA   
4  1003000282                    BLAKEMORE                     ROSIE   

  nppes_provider_mi nppes_credentials nppes_provider_gender nppes_entity_code  \
0               NaN              M.D.                     M                 I   
1               NaN              M.D.                     M                 I   
2                 E               DDS                     M                 I   
3               NaN            D.D.S.                     F                 I   
4                 K               FNP                     F                 I   

       nppes_provider_street1 nppes_provider_street2 nppes_provider_city  ...  \

In 2017, what was the average number of beneficiaries per provider? Due to the suppression of data for those with few beneficiaries, we can only include those with more than 10 beneficiaries.

In [3]:
data1 = data['bene_count']
data1 = data1.dropna()
print(data1)
total = np.sum(data1.values)
print(total/1162897)

0          234.0
1          276.0
2           33.0
4           11.0
5          117.0
           ...  
1162891     37.0
1162892     19.0
1162893    144.0
1162896     38.0
1162897     77.0
Name: bene_count, Length: 1031058, dtype: float64
140.3971942485018


For each provider, estimate the length of the average prescription from the total_day_supply and total_claim_count. What is the median, in days, of the distribution of this value across all providers?

In [4]:
data2 = data[['total_claim_count','total_day_supply']]
data2 = data2.dropna()
data2 = data['total_claim_count']/data['total_day_supply']
print(data2.head())
print(data2.describe())

0    0.045780
1    0.033205
2    0.127907
3    0.120000
4    0.033570
dtype: float64
count    1.162898e+06
mean     5.543928e-02
std      6.880941e-02
min      3.908286e-03
25%      2.352941e-02
50%      3.365578e-02
75%      6.490872e-02
max      1.000000e+00
dtype: float64


Work out for each Specialty the fraction of drug claims that are for brand-name drugs. Include only providers for whom the relevant information has not been suppressed, and consider only specialties with at least 1000 total claims. What is the standard deviation of these fractions?

In [5]:
data3_drop = data[['specialty_description','brand_claim_count']].dropna()
data3_group = data3_drop.groupby(by='specialty_description').sum()
data3_count = data3_drop['specialty_description'].value_counts()
data3_concat = pd.concat([data3_group['brand_claim_count'],data3_count],axis=1,sort=True, keys=['brand_claim_count','count'])
data3_1000 = data3_concat[data3_concat['count']>1000]
data3_frac = data3_1000['brand_claim_count']/data3_1000['count']
print(data3_1000)
print(data3_frac)
print(np.std(data3_frac.values))

                                                    brand_claim_count  count
Allergy/ Immunology                                          883748.0   3457
Anesthesiology                                               503470.0   3913
Cardiology                                                 10222462.0  16140
Certified Clinical Nurse Specialist                          303049.0   1785
Clinical Cardiac Electrophysiology                           829875.0   1854
Critical Care (Intensivists)                                 470162.0   1071
Dentist                                                       28921.0  85976
Dermatology                                                  948739.0  10574
Diagnostic Radiology                                          43881.0   1916
Emergency Medicine                                          1645244.0  22349
Endocrinology                                               7076382.0   5532
Family Practice                                            63474840.0  76356

Find the ratio of beneficiaries with opioid prescriptions to beneficiaries with antibiotics prescriptions in each state. Assume that each beneficiary attends only a single provider. What is the difference between the largest and smallest ratios?

In [6]:
data4 = data[['nppes_provider_state','opioid_bene_count','antibiotic_bene_count']]
data4 = data4.dropna()
data4 = data4.groupby(by='nppes_provider_state').sum()
data4 = data4['opioid_bene_count']/data4['antibiotic_bene_count']
print(data4.describe())
print(data4.max()-data4.min())

count    61.000000
mean      0.621862
std       0.113623
min       0.358576
25%       0.557738
50%       0.638357
75%       0.694272
max       0.860136
dtype: float64
0.501559277375204


For each provider where the relevant columns are not suppressed, work out the fraction of claims for beneficiaries age 65 and older, as well as the fraction of claims for beneficiaries with a low-income subsidy. What is the Pearson correlation coefficient between these values?

In [7]:
from scipy.stats import pearsonr
data5 = data[['total_claim_count_ge65','lis_claim_count']]
data5 = data5.dropna()
print(data5)
pearson = pearsonr(data5['total_claim_count_ge65'].values, data5['lis_claim_count'].values)
print(pearson)

         total_claim_count_ge65  lis_claim_count
0                         516.0            305.0
1                         881.0           1184.0
4                          65.0             73.0
5                        2700.0           1990.0
6                         118.0             70.0
...                         ...              ...
1162891                    94.0            145.0
1162893                   229.0             58.0
1162894                    59.0              0.0
1162896                    66.0             28.0
1162897                    54.0            220.0

[834684 rows x 2 columns]
(0.8067444425923471, 0.0)


Let's find which states have surprisingly high supply of opioids, conditioned on specialty. Work out the average length of an opioid prescription for each provider. For each (state, specialty) pair with at least 100 providers, calculate the average of this value across all providers. Then find the ratio of this value to an equivalent quantity calculated from providers in each specialty across all states. What is the largest such ratio?

In [8]:
data6_select = data[['nppes_provider_state','specialty_description','opioid_claim_count']]
data6_select = data6_select.dropna()
data6 = data6_select.groupby(['nppes_provider_state','specialty_description']).agg({'opioid_claim_count':[np.mean, 'count']})
data6 = data6[data6['opioid_claim_count']['count']>100]
print(data6)
data6 = data6['opioid_claim_count']['mean']/data6['opioid_claim_count']['count']
print(data6.describe())

                                                                        opioid_claim_count  \
                                                                                      mean   
nppes_provider_state specialty_description                                                   
AK                   Family Practice                                             63.839394   
                     Nurse Practitioner                                          51.912000   
                     Physician Assistant                                         60.697581   
AL                   Anesthesiology                                            1314.533981   
                     Cardiology                                                  27.829060   
...                                                                                    ...   
WV                   Psychiatry                                                  11.834783   
                     Student in an Organized Health Care Edu

For each provider for whom the information is not suppressed, figure out the average cost per day of prescriptions in both 2016 and 2017. Use this to estimate the inflation rate for daily prescription costs per provider. What is the average inflation rate across all providers?

In [ ]:
data_16 = pd.read_csv('PartD_Prescriber_PUF_NPI_16.txt',sep='\t')
print(data.head())

In [ ]:
year_17 = data[['total_drug_cost','bene_count']]
year_16 = data_16[['total_drug_cost','bene_count']]
total_year = pd.concat([year_17,year_16],axis=1,sort=True, keys=['2017','2016'])
total_year = total_year.dropna()
print(total_year)
year_17 = total_year['2017']['total_drug_cost']/365
year_16 = total_year['2016']['total_drug_cost']/366
inflate = ((year_17 - year_16)/year_16)*100
print(np.mean(inflate.values))
print(inflate.describe())

Consider all providers with a defined specialty in both years. Find the fraction of providers who left each specialty between 2016 and 2017. What is the largest such fraction, when considering specialties with at least 1000 proviers in 2016? Note that some specialties have a fraction of 1 due to specialty name changes between 2016 and 2017; disregard these specialties in calculating your answer.

In [ ]:
count_17 = data['specialty_description'].replace(['Allergy/ Immunology','Anesthesiologist Assistants','Audiologist (billing independently)','Behavioral Analyst','Cardiovascular Disease (Cardiology)','Certified Registered Nurse Anesthetist (CRNA)','Clinical Cardiatric Electrophysiology','Hospital (Dmercs Only)','Hospitalist','Midwife, Lay','Obstetrics & Gynecology','Occupational Therapist in Private Practice','Occupational Therapy Assistant','Oral Surgery (Dentist only)','Pharmacy','Physical Therapist in Private Practice','Prosthetic/Orthotic Supplier','Psychologist (billing independently)','Psychologist, Clinical','Public Health Welfare Agency','Radiation Therapy','Registered Dietician/Nutrition Professional','SNF (Dmercs Only)','Spec/Tech, Cardiovascular','Spec/Tech, Health Info','Spec/Tech, Pathology','Voluntary Health or Charitable Agencies'],['Allergy/Immunology','Anesthesiology Assistant','Audiologist','Behavior Analyst','Cardiology','CRNA','Clinical Cardiac Electrophysiology','Hospital','Hospital','Midwife','Obstetrics/Gynecology','Occupational therapist','Occupational therapist','Oral Surgery (Dentists only)','Pharmacist','Physical Therapist','Prosthetist','Psychologist','Psychologist','Public Health or Welfare Agency','Radiation Therapy Center','Registered Dietitian or Nutrition Professional','Skilled Nursing Facility','Specialist/Technologist Cardiovascular','Specialist/Technologist, Other','Specialist/Technologist, Pathology','Voluntary Health or Charitable Agency'])
count_16 = data_16['specialty_description'].replace(['Allergy/ Immunology','Anesthesiologist Assistants','Audiologist (billing independently)','Behavioral Analyst','Cardiovascular Disease (Cardiology)','Certified Registered Nurse Anesthetist (CRNA)','Clinical Cardiatric Electrophysiology','Hospital (Dmercs Only)','Hospitalist','Midwife, Lay','Obstetrics & Gynecology','Occupational Therapist in Private Practice','Occupational Therapy Assistant','Oral Surgery (Dentist only)','Pharmacy','Physical Therapist in Private Practice','Prosthetic/Orthotic Supplier','Psychologist (billing independently)','Psychologist, Clinical','Public Health Welfare Agency','Radiation Therapy','Registered Dietician/Nutrition Professional','SNF (Dmercs Only)','Spec/Tech, Cardiovascular','Spec/Tech, Health Info','Spec/Tech, Pathology','Voluntary Health or Charitable Agencies'],['Allergy/Immunology','Anesthesiology Assistant','Audiologist','Behavior Analyst','Cardiology','CRNA','Clinical Cardiac Electrophysiology','Hospital','Hospital','Midwife','Obstetrics/Gynecology','Occupational therapist','Occupational therapist','Oral Surgery (Dentists only)','Pharmacist','Physical Therapist','Prosthetist','Psychologist','Psychologist','Public Health or Welfare Agency','Radiation Therapy Center','Registered Dietitian or Nutrition Professional','Skilled Nursing Facility','Specialist/Technologist Cardiovascular','Specialist/Technologist, Other','Specialist/Technologist, Pathology','Voluntary Health or Charitable Agency'])
count_17 = count_17.value_counts()
count_16 = count_16.value_counts()
count_year = pd.concat([count_17,count_16],axis=1,sort=True, keys=['2017','2016'])
count_year = count_year.replace(np.nan,0)
count_year = count_year[count_year['2016']>1000]
left = count_year['2017']-count_year['2016']
final_data = left[left<0]/count_year['2016']
print(final_data.describe())
